In [252]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [253]:
driver = webdriver.Firefox()

In [254]:
driver.get("https://consumidor.gov.br/pages/indicador/geral/abrir")

In [255]:
wait = WebDriverWait(driver, 10)

In [256]:
wait.until(EC.element_to_be_clickable((By.ID, 'li_tab_relatos')))

<selenium.webdriver.remote.webelement.WebElement (session="74ed6f68-c6ea-457d-abc8-9fef9401f71b", element="1f8c923b-8a48-4ea8-acea-4773ddeeee8f")>

In [257]:
tab_relatos = driver.find_element(By.ID, "li_tab_relatos")

In [258]:
tab_relatos.click()

In [259]:
btn_mais_resultados = driver.find_element(By.ID, "btn-mais-resultados")

In [260]:
btn_mais_resultados.click()

In [261]:
resultados = driver.find_element(By.ID, 'resultados')

In [262]:
nom_empresas = resultados.find_elements(By.CLASS_NAME, 'relatos-nome-empresa')


In [263]:
empresas = []

In [264]:
for empresa in nom_empresas:
    empresas.append(empresa.find_element(By.TAG_NAME, 'a').text)    

In [265]:
rel_status = resultados.find_elements(By.TAG_NAME, 'h4')

In [266]:
lst_status = []

In [267]:
for status in rel_status:
    lst_status.append(status.text)        

In [268]:
rel_datas = resultados.find_elements(By.CLASS_NAME, 'relatos-data')

In [269]:
tempo_respostas = []
datas = []
cidades = []
ufs = []
count = 1

In [270]:
for data in rel_datas:
    if count % 2 == 0:
        tempo_respostas.append(data.text.replace('(','').replace(')',''))   
    else:
        datas.append(data.text.split(',')[0])
        cidades.append(data.text.split(',')[1].split('-')[0].strip())
        ufs.append(data.text.split(',')[1].split('-')[1].strip())
    count=count+1

In [271]:
lst_relatos = resultados.find_elements(By.XPATH, "//div[3]/p")
lst_respostas = resultados.find_elements(By.XPATH, "//div[4]/p")
lst_notas_comentarios = resultados.find_elements(By.XPATH, "//div[5]/p")

In [272]:
relatos = []
respostas = []
notas = []
comentarios = []

In [273]:
for relato in lst_relatos:
    relatos.append(relato.text)        

In [274]:
for resposta in lst_respostas:
    respostas.append(resposta.text)     

In [275]:
count = 1
for nota in lst_notas_comentarios:
    if count % 2 == 0:
        comentarios.append(nota.text)
    else:
        notas.append(nota.text)
    count=count+1

In [276]:
relatos = list(zip(empresas,lst_status,tempo_respostas, datas, cidades, ufs, relatos, respostas, notas, comentarios))

In [277]:
relatos

[('Cemig',
  'Resolvido',
  'no mesmo dia',
  '10/10/2024',
  'Ibirité',
  'MG',
  'Quero saber quais são os canais digitais da Cemig.',
  'Olá, Ketlen ! Boa Noite, Tudo bem? Em resposta a sua manifestação no site consumidor.gov, SERVIÇOS POR CANAL DE ATENDIMENTO Conheça os canais de atendimento da Cemig e os serviços disponíveis em cada um. Pensando na proteção e comodidade dos clientes, os nossos serviços estão disponíveis nos canais digitais de atendimento. Confira abaixo o passo a passo para acessá-los. CANAIS DIGITAIS DE ATENDIMENTO: Cemig Atende Web App Cemig Atende – disponível para Android e iOS. WhatsApp: (31) XXXXX-X160 Telegram: @cemigbot Central de atendimento 116: prioridade para emergências (alguns serviços continuam disponíveis no atendimento eletrônico). Agendamento para atendimento presencial ou vídeo atendimento: Reiteramos que permanecemos à disposição para quaisquer esclarecimentos que se façam necessários também nos canais de atendimento abaixo: Cemig Atende APP Wh

In [279]:
driver.close()